### part 1

In [11]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.

from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * (2.0 * pi)
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            # square error
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            # sensor noise
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')         
        
        # turn, and add randomness to the turning command
        # noise could be pos or neg
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        # mod
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        # toroidal
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        # instantiate a new robot each time it moves?
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))



####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.

measurements=[]
myrobot = robot()
myrobot.set(30,50,pi/2.0)
myrobot.set_noise(5.0, 0.1, 5.0)
myrobot = myrobot.move(-pi/2.0,15)
measurements.append(myrobot.sense())
myrobot = myrobot.move(-pi/2.0, 10)
myrobot.set_noise(5.0,0.1,5.0)
measurements.append(myrobot.sense())

for i in range(len(measurements)):
    print("{}\n".format(measurements[i]))

# Now add noise to your robot as follows:
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.

# Now we want to create particles,
# p[i] = robot(). In this assignment, write
# code that will assign 1000 such particles
# to a list.

N = 1000
p = []

for j in range(1000):
    p.append(robot())

print(len(p))
print(p[:10])

# Now we want to simulate robot
# motion with our particles.
# Each particle should turn by 0.1
# and then move by 5. 
new_p = []

for k in range(len(p)):
    new_p.append(p[k].move(0.1, 5.0))
    
print("\nnew_p\n {}\n".format(new_p[:10]))

[35.105864212968754, 51.50613092199062, 44.745928068233866, 46.48118422623672]

[36.470011161154545, 53.16959659535933, 32.9279641815219, 44.10321457755051]

1000
[[x=38.361 y=11.458 orient=0.5973], [x=36.382 y=78.601 orient=2.2282], [x=51.006 y=8.8365 orient=2.2839], [x=28.191 y=40.177 orient=1.7931], [x=81.000 y=44.661 orient=2.5371], [x=31.504 y=95.938 orient=0.3067], [x=4.9072 y=2.8580 orient=4.0657], [x=26.599 y=94.400 orient=2.5305], [x=59.197 y=88.120 orient=1.5633], [x=92.371 y=41.552 orient=0.6717]]

new_p
 [[x=42.193 y=14.668 orient=0.6973], [x=32.946 y=82.233 orient=2.3282], [x=47.373 y=12.272 orient=2.3839], [x=26.607 y=44.919 orient=1.8931], [x=76.623 y=47.078 orient=2.6371], [x=36.096 y=97.916 orient=0.4067], [x=2.3082 y=98.586 orient=4.1657], [x=22.238 y=96.845 orient=2.6305], [x=58.734 y=93.098 orient=1.6633], [x=95.955 y=45.039 orient=0.7717]]



### part 2 weight calculation

In [43]:
# Now we want to give weight to our 
# particles. This program will print a
# list of 1000 particle weights.
#
# Don't modify the code below. Please enter
# your code at the bottom.

from math import *
import random


landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        '''
        simulates imperfect sensing of distance to each landmark
        return: array of 4 distances
        '''
        Z = []
        for i in range(len(landmarks)):
            #Euclidean distance
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            #add Gaussian noise
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')         
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        # measurement is array of 4, distance to each landmark
        # calculates how likely a measurement should be, prob of measurement in Gaussian/normal distrib
        # which tells us whether it should be kept or is total garbage so chuck it in resampling
        
        prob = 1.0;
        for i in range(len(landmarks)):
            # Euclidan distance
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            # prob = Gaussian(mu, sigma, x)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
     
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))


#myrobot = robot()
#myrobot.set_noise(5.0, 0.1, 5.0)
#myrobot.set(30.0, 50.0, pi/2)
#myrobot = myrobot.move(-pi/2, 15.0)
#print myrobot.sense()
#myrobot = myrobot.move(-pi/2, 10.0)
#print myrobot.sense()

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
# Now we want to give weight to our 
# particles.Ie.  How likely is this particle where the robot is based on the measurements? 
# Gaussian's mean is the actual distance
#This program will print a
# list of 1000 particle weights.

#insert code here!

for j in range(N):
    prob = p[j].measurement_prob(p[j].sense())
    w.append(prob)

print (w) #Please print w for grading purposes.








[7.555554739063689e-07, 2.702788526226119e-06, 1.2822359980001434e-05, 3.0151888385085578e-05, 9.303494942161974e-06, 3.7082458659823653e-06, 4.4987147652444985e-06, 2.603525468699213e-05, 8.043588469677624e-06, 3.2299548383758566e-05, 9.67164185248579e-07, 1.1826047023616326e-05, 6.3880959385631255e-06, 2.657906147410525e-06, 2.1260082815904904e-05, 3.537340777231957e-06, 1.0008659422292929e-05, 2.7458042486707417e-05, 2.1712069262809916e-05, 1.9800143209084305e-06, 3.4358005077957536e-06, 7.933732112767381e-06, 2.8637646843432366e-06, 4.429255516776601e-06, 1.1363927246086094e-05, 1.9894991781305176e-05, 7.237194605842549e-06, 3.351779831173281e-05, 7.668269507750901e-06, 2.6772879046960247e-05, 1.766295626779873e-05, 1.1938120259593118e-05, 3.7307324307438917e-06, 1.8590320849820646e-05, 2.4865888898874093e-08, 3.617760874578923e-06, 1.0680369846944384e-06, 3.1771281498331246e-06, 2.328915065892852e-05, 1.0338553455769736e-05, 1.054039023704838e-05, 1.352238957042621e-06, 3.37927877

### part 3 resampling wheel 

This actually is a "simpler" calculation than to normalize weights and to choose based on that.  A trick where the w's are added to make wheel that totals to more than 1 which in effect normalizes it.  

In [53]:
# In this exercise, try to write a program that
# will resample particles according to their weights.
# Particles with higher weights should be sampled
# more frequently (in proportion to their weight).

# Don't modify anything below. Please scroll to the 
# bottom to enter your code.

from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')         
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))


#myrobot = robot()
#myrobot.set_noise(5.0, 0.1, 5.0)
#myrobot.set(30.0, 50.0, pi/2)
#myrobot = myrobot.move(-pi/2, 15.0)
#print myrobot.sense()
#myrobot = myrobot.move(-pi/2, 10.0)
#print myrobot.sense()

myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
for i in range(N):
    w.append(p[i].measurement_prob(Z))


#### DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
# You should make sure that p3 contains a list with particles
# resampled according to their weights.
# Also, DO NOT MODIFY p.
# sum w to get W, normalize w[i..N] by dividing each by W
# randomly based on w[i..N] pick and then use index to pick from p
# append to p3[i..N]

'''
# my way, Python3
p3 = []
W = sum(w)
w_norm = []
for i in range(N):
    w_norm.append(w[i]/W)

p3 = random.choices(population=p, weights=w_norm, k=N)
'''
# resampling wheel method to resample particles based on the assigned weights
p3 = []
idx = int(random.random() * N)
#print("idx", idx)
beta = 0
for i in range(N):
    # get size of beta
    beta = (random.random() * (2 * max(w)))
    # get value of beta by subtracting up to w_norm[idx]
    while(w[idx] < beta):
        beta -= w[idx]
        idx = (idx+1)%N
        
    p3.append(p[idx])
p = p3
print(p)



[[x=91.703 y=69.893 orient=5.6922], [x=96.372 y=71.700 orient=3.3959], [x=91.973 y=69.568 orient=2.8975], [x=91.973 y=69.568 orient=2.8975], [x=93.649 y=70.498 orient=1.2725], [x=97.824 y=67.658 orient=2.8727], [x=91.427 y=66.923 orient=0.0504], [x=94.507 y=64.486 orient=1.6071], [x=94.381 y=63.588 orient=0.8146], [x=90.900 y=66.236 orient=5.5405], [x=92.620 y=64.972 orient=5.5179], [x=91.703 y=69.893 orient=5.6922], [x=91.703 y=69.893 orient=5.6922], [x=99.244 y=69.653 orient=2.0607], [x=96.372 y=71.700 orient=3.3959], [x=99.232 y=70.361 orient=4.0332], [x=93.649 y=70.498 orient=1.2725], [x=91.427 y=66.923 orient=0.0504], [x=94.507 y=64.486 orient=1.6071], [x=97.063 y=69.183 orient=3.1955], [x=90.179 y=66.509 orient=1.1007], [x=91.703 y=69.893 orient=5.6922], [x=96.372 y=71.700 orient=3.3959], [x=96.372 y=71.700 orient=3.3959], [x=91.973 y=69.568 orient=2.8975], [x=91.973 y=69.568 orient=2.8975], [x=91.973 y=69.568 orient=2.8975], [x=93.649 y=70.498 orient=1.2725], [x=91.427 y=66.923 